# Inferring

Goal of this notebook is practicing and understanding prompts used for inferring.

Examples sourced from [Deeplearning.ai ChatGPT Prompt Engineering for Developers](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/5/inferring) course.

LLM Model: [dolly-v2-3b](https://huggingface.co/databricks/dolly-v2-3b)

In [1]:
!export PYTORCH_ENABLE_MPS_FALLBACK=1
import time

import torch
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# print(torch.backends.mps.is_available())
# print(torch.backends.mps.is_built())
# print(torch.cuda.is_available())

In [2]:
generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map="auto", return_full_text=True)

simple_prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}")

prompt_with_context = PromptTemplate(
    input_variables=["instruction", "context"],
    template="{instruction}\n\nInput:\n{context}")

hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

llm_chain = LLMChain(llm=hf_pipeline, prompt=simple_prompt)
llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context)


def get_response(input_prompt, context=None):
    if context is None:
        return llm_chain.predict(instruction=input_prompt).lstrip()
    else:
        return llm_context_chain.predict(instruction=input_prompt, context=context).lstrip()


def reply_prompt(input_prompt, context=None):
    start = time.time()
    response = get_response(input_prompt, context)
    end = time.time()
    duration_seconds = end - start
    print(response)
    print(f"\nProcessing duration: {duration_seconds:0.2f} seconds")


## Challenge: infer sentiment and topics from product reviews and news articles.

In [3]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had additional storage and not too high of a price point.
Got it fast.
The string to our lamp broke during the transit and the company happily sent over a new one.
Came within a few days as well.
It was easy to put together.
I had a missing part, so I contacted their support and they very quickly got me the missing piece!
Lumina seems to me to be a great company that cares about their customers and products!!
"""

### Sentiment: Positive / Negative

In [4]:
prompt = f"""
What is the sentiment of the following product review which is delimited with <>?
Give your answer as a single word, either "positive" or "negative".
Review text: <{lamp_review}>
"""

reply_prompt(prompt)

positive

Processing duration: 108.44 seconds


### Identify types of emotions

In [14]:
prompt = f"""
1 - Identify a list of emotions that the writer of the following review review which is delimited by <>.
2 - Format your answer as a list of words.
3 - Include no more than five items in the list.

Review text: <{lamp_review}>
"""

reply_prompt(prompt)

- Happiness
- Ease of buying
- Good price
- Good company
- Missing part
- String broke

Processing duration: 131.35 seconds


### Identify anger

In [6]:
prompt = f"""
Is the writer of the following review expressing anger?
The review is delimited with <>.
Give your answer as either yes or no.

Review text: <{lamp_review}>
"""

reply_prompt(prompt)

No, the reviewer is expressing gratitude.

Processing duration: 111.45 seconds


### Extract product and company names

In [13]:
prompt = f"""
Identify the following items from the review text:
- Item purchased by reviewer
- Company that made the item

The review is delimited with <>.
Format your response as a JSON object with "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" as the value.
Make your response as short as possible.

Review text: <{lamp_review}>
"""

reply_prompt(prompt)

"Item": "Lumina", "Brand": "Lumina"

Processing duration: 142.63 seconds


### Doing multiple tasks at once

In [12]:
prompt = f"""
Identify the following items from the review text:
1 - Sentiment (positive or negative)
2 - Is the reviewer expressing anger? (true or false)
3 - Item purchased by reviewer
4 - Company that made the item

The review is delimited with <>.
Format your response as a JSON object with the following keys:  sentiment, anger_detected, item, brand.
If the information isn't present, use "unknown" as the value.
Make your response as short as possible.
Format the anger_detected value as a boolean.

Review text: <{lamp_review}>
"""

reply_prompt(prompt)

{
  "sentiment": "positive",
  "anger_detected": false,
  "item": "Lumina",
  "brand": "Company that made the item"
}

Processing duration: 190.05 seconds


---
## Inferring topics

In [9]:
story = """
In a recent survey conducted by the government, public sector employees were asked to rate their level of satisfaction with the department they work at.
The results revealed that NASA was the most popular department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, stating, "I'm not surprised that NASA came out on top.
It's a great place to work with amazing people and incredible opportunities. I'm proud to be a part of such an innovative organization."

The results were also welcomed by NASA's management team, with Director Tom Johnson stating, "We are thrilled to hear that our employees are satisfied with their work at NASA.
We have a talented and dedicated team who work tirelessly to achieve our goals, and it's fantastic to see that their hard work is paying off."

The survey also revealed that the Social Security Administration had the lowest satisfaction rating, with only 45% of employees indicating they were
satisfied with their job. The government has pledged to address the concerns raised by employees in the survey and work towards improving job satisfaction across all departments.
"""

### Infer 5 topics

In [10]:
prompt = f"""
Determine five topics that are being discussed in the following text, which is delimited by <>.

Make each item at most 30 characters long.

Format your response as a list of items.

Text sample: <{story}>
"""

reply_prompt(prompt)

- NASA
- Social Security Administration
- The government
- John Smith
- Director Tom Johnson

Processing duration: 196.99 seconds
